In [1]:
import requests
import cv2
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Flatten


In [2]:
# fetch images and categories
%matplotlib inline
BASE_URL = "http://192.168.31.19:8080/storage_management"
CATEGORY = "category/"
ITEM_IMAGES = "item-image/"


In [3]:
categories: [] = requests.get(f"{BASE_URL}/{CATEGORY}").json()
item_images = requests.get(f"{BASE_URL}/{ITEM_IMAGES}").json()


In [4]:
def get_category_name(cs: [], cid: int) -> str:
    for c in cs:
        if c['id'] == cid:
            return c['name']


In [5]:
num_categories = len(categories)
image_urls = [i['image'] for i in item_images]
labels = np.asarray([i['category']['id'] - 1 for i in item_images])
image_paths = [os.path.basename(i['image']) for i in item_images]
print(image_urls[0])
print(labels[0])


http://192.168.31.19:8080/media/item-image/2019-07-12_033125.213847.jpg
1


In [6]:
from tqdm import tqdm
import shutil
import os
# download images
for i, url in enumerate(tqdm(image_urls)):
    res = requests.get(url, stream=True)
    path = os.path.join(os.getcwd(), f"images/{os.path.basename(url)}")
    with open(path, "wb") as out_file:
        shutil.copyfileobj(res.raw, out_file)
    del res


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:00<00:08,  2.71it/s]

  8%|▊         | 2/24 [00:00<00:07,  2.90it/s]

 12%|█▎        | 3/24 [00:00<00:07,  2.91it/s]

 17%|█▋        | 4/24 [00:01<00:06,  2.99it/s]

...

 83%|████████▎ | 20/24 [00:06<00:01,  2.79it/s]

 88%|████████▊ | 21/24 [00:06<00:01,  2.89it/s]

 92%|█████████▏| 22/24 [00:07<00:00,  2.39it/s]

 96%|█████████▌| 23/24 [00:07<00:00,  2.70it/s]

100%|██████████| 24/24 [00:07<00:00,  2.83it/s]

In [7]:

# load images
WIDTH = 224
HEIGHT = 224
images = []
for path in image_paths:
    path = os.path.join(f"images/{path}")
    image = tf.keras.preprocessing.image.load_img(path, target_size=(WIDTH, HEIGHT))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.keras.applications.vgg19.preprocess_input(image)
    images.append(image)
images = np.asarray(images)
images.shape

(24, 224, 224, 3)

In [26]:
# create model
base_model = tf.keras.applications.VGG19(weights='imagenet',
                                         include_top=False,
                                         input_shape=(WIDTH, HEIGHT, 3))
base_model.trainable = True
model = tf.keras.Sequential([base_model, tf.keras.layers.GlobalAveragePooling2D()])
model.add(Dense(num_categories, activation="softmax"))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [76]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 3078      
Total params: 20,027,462
Trainable params: 3,078
Non-trainable params: 20,024,384
_________________________________________________________________


In [27]:
# print(images.shape)
# print(labels.shape)
model.fit(images, labels, epochs=6) 


Epoch 1/6


24/24 [==============================] - 8s 313ms/sample - loss: 10.1256 - acc: 0.1250


Epoch 2/6


24/24 [==============================] - 8s 318ms/sample - loss: 790.6265 - acc: 0.6667


Epoch 3/6


...

24/24 [==============================] - 8s 327ms/sample - loss: 104.4095 - acc: 0.6667


Epoch 5/6


24/24 [==============================] - 8s 325ms/sample - loss: 271.8305 - acc: 0.0833


Epoch 6/6


24/24 [==============================] - 8s 328ms/sample - loss: 2.5332 - acc: 0.6667


In [30]:
test_image = cv2.imread("test.jpg")
test_image = cv2.resize(test_image, (WIDTH, HEIGHT))


In [31]:
pre = model.predict(test_image.reshape(1, WIDTH, HEIGHT, 3))[0]
print(pre)
index = np.argmax(pre) + 1
print(get_category_name(categories, index))


[0.00128887 0.25411215 0.73122054 0.01069771 0.00268068]
蓝光碟
